# Chapter 1: Initial Setup and Dependencies

In [1]:
!pip install uv


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: C:\Users\itsme\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


# Chapter 2: Working with text data

In [4]:
text = '''
## Attention Is All You Need: A Paradigm Shift in Sequence Modeling

The year 2017 marked a pivotal moment in the history of deep learning with the publication of the paper "Attention Is All You Need" by Vaswani et al. from Google Brain. This seminal work introduced the **Transformer** architecture, which completely revolutionized the field of **sequence modeling**, particularly in natural language processing (NLP). Prior to the Transformer, recurrent neural networks (RNNs) and their variants like LSTMs and GRUs were the dominant architectures for tasks involving sequential data. While effective, these models suffered from inherent limitations, most notably their sequential nature, which hindered parallelization and long-range dependency modeling. The Transformer, with its reliance solely on **attention mechanisms**, elegantly sidestepped these issues, paving the way for unprecedented advancements.

### The Limitations of Recurrent Architectures

Before delving into the brilliance of the Transformer, it's crucial to understand the challenges faced by its predecessors. RNNs process sequences one element at a time, feeding the hidden state from the previous step into the current one. This sequential dependency, while seemingly intuitive for sequences, created several bottlenecks:

* **Lack of Parallelization:** Because each step depends on the previous one, RNNs cannot be fully parallelized during training. This significantly slowed down training times, especially for very long sequences.
* **Vanishing/Exploding Gradients:** While LSTMs and GRUs alleviated this to some extent, the problem of vanishing or exploding gradients still made it difficult for RNNs to capture long-range dependencies effectively. Information from early parts of a long sequence could easily be diluted or lost by the time it reached the later parts.
* **Fixed Context Window (Implicitly):** Although LSTMs have "memory cells," their ability to retain information across very long distances was still limited, making it challenging to model relationships between distant words in a sentence or across paragraphs.

### The Rise of Attention: A Glimmer of Hope

The concept of **attention** wasn't entirely new in 2017. It had already been introduced and successfully applied in conjunction with RNNs, notably in machine translation tasks. In these "encoder-decoder" RNN models with attention, the attention mechanism allowed the decoder to selectively focus on different parts of the input sequence when generating an output. This was a significant improvement, enabling the model to "attend" to relevant input tokens, rather than trying to compress all information into a single fixed-size context vector. However, attention was still an add-on, enhancing an underlying recurrent structure.

### The Radical Idea: Attention Is All You Need

The true genius of the Transformer lay in its audacious claim: **recurrent and convolutional layers were unnecessary**. The entire architecture could be built purely on attention mechanisms. This was a radical departure, as convolutions and recurrences had been the bedrock of deep learning for sequence and image processing, respectively. The paper demonstrated that attention, when properly configured, could not only match but significantly surpass the performance of existing models on challenging tasks like machine translation.

The core innovation of the Transformer is the **self-attention mechanism**, specifically **multi-head self-attention**. Instead of attending to an external sequence (as in encoder-decoder attention), self-attention allows each element in a sequence to attend to all other elements within the *same* sequence. This enables the model to weigh the importance of different words in understanding the meaning of a given word within its context. For example, in the sentence "The animal didn't cross the street because it was too tired," self-attention helps the model understand that "it" refers to "the animal."

### Deconstructing the Transformer Architecture

The Transformer architecture consists of an **encoder** and a **decoder**, each composed of multiple identical layers.

#### Encoder:
Each encoder layer has two sub-layers:
1.  **Multi-Head Self-Attention:** This is where the magic happens. It allows the model to capture dependencies between different words in the input sequence. "Multi-head" means that the attention mechanism is run multiple times in parallel, each with different learned linear projections, allowing the model to attend to information from different representation subspaces at different positions. The outputs of these attention heads are then concatenated and linearly transformed.
2.  **Feed-Forward Network:** A simple position-wise fully connected feed-forward network is applied independently to each position.

Crucially, **residual connections** and **layer normalization** are used around each of these sub-layers. Residual connections help with gradient flow and enable deeper networks, while layer normalization stabilizes training.

#### Decoder:
The decoder also has multiple identical layers, but with an additional sub-layer:
1.  **Masked Multi-Head Self-Attention:** Similar to the encoder's self-attention, but it's "masked" to prevent positions from attending to subsequent positions. This ensures that the prediction for a given position can only depend on known outputs at previous positions.
2.  **Multi-Head Encoder-Decoder Attention:** This layer allows the decoder to attend to the output of the encoder. It works similarly to the attention mechanisms used in pre-Transformer RNN-based encoder-decoder models.
3.  **Feed-Forward Network:** Same as in the encoder.

#### Positional Encoding:
Since the Transformer has no recurrence or convolution, it has no inherent notion of word order. To inject this sequential information, **positional encodings** are added to the input embeddings. These are fixed (sinusoidal functions) or learned embeddings that provide information about the absolute or relative position of each token in the sequence.

### The Impact and Legacy

The implications of "Attention Is All You Need" were profound and far-reaching:

* **Superior Performance:** Transformers quickly achieved state-of-the-art results across a wide range of NLP tasks, from machine translation and text summarization to question answering and sentiment analysis.
* **Parallelization and Scalability:** The non-sequential nature of the Transformer allowed for unprecedented parallelization during training, enabling the development of much larger and more powerful models. This was a critical factor in the rise of large pre-trained language models.
* **Foundation for Large Language Models (LLMs):** The Transformer architecture is the bedrock of virtually all modern LLMs, including BERT, GPT (GPT-2, GPT-3, GPT-4, etc.), T5, LLaMA, and many more. These models, pre-trained on massive datasets, have demonstrated remarkable capabilities in understanding and generating human language, driving the current AI revolution.
* **Transfer Learning:** The ability of Transformers to learn rich, contextualized representations made them ideal for **transfer learning**. Models pre-trained on vast amounts of text data could then be fine-tuned on smaller, task-specific datasets with excellent results, significantly reducing the need for large labeled datasets.
* **Beyond NLP:** While initially designed for NLP, the success of Transformers has extended to other domains, including computer vision (e.g., Vision Transformers for image recognition), speech processing, and even reinforcement learning. This demonstrates the versatility and generalizability of the attention mechanism.

### Challenges and Future Directions

Despite its immense success, the Transformer architecture is not without its challenges:

* **Computational Cost for Long Sequences:** The self-attention mechanism has a quadratic complexity with respect to the sequence length, meaning that as sequences get very long, the computational cost (both memory and processing time) becomes prohibitive. This is an active area of research, with efforts to develop more efficient attention mechanisms (e.g., sparse attention, linear attention, attention with local windows).
* **Interpretability:** While attention weights can sometimes offer insights into what the model is focusing on, interpreting the complex interplay of multiple attention heads and deep layers remains a significant challenge.
* **Bias:** Large language models built on Transformers can inherit and amplify biases present in their training data, leading to unfair or harmful outputs. Addressing these biases is a critical ongoing effort.
* **Energy Consumption:** Training and running these massive Transformer models require substantial computational resources and energy, raising concerns about their environmental impact.

### Conclusion

"Attention Is All You Need" didn't just introduce a new architecture; it fundamentally reshaped our understanding of how to process sequential data in deep learning. By demonstrating the power and efficiency of attention mechanisms alone, it unleashed a wave of innovation that continues to drive progress in artificial intelligence. The Transformer's elegance, scalability, and ability to capture intricate dependencies have made it the de facto standard for sequence modeling, laying the groundwork for the era of large language models and pushing the boundaries of what machines can understand and generate. The journey is far from over, with researchers continuously refining and extending the Transformer's capabilities, ensuring that attention remains at the forefront of AI research for the foreseeable future.

---'''

In [4]:
with open('sample_data.txt', 'w') as f:
    f.write(text)

In [5]:
with open("sample_data.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 9728

## Attention Is All You Need: A Paradigm Shift in Sequence Modeling

The year 2017 marked a pivota


In [6]:
import re

text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)

print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


In [7]:
result = re.split(r'([,.]|\s)', text)

print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [8]:
# Strip whitespace from each item and then filter out any empty strings.
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


In [9]:
text = "Hello, world. Is this-- a test?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [10]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['##', 'Attention', 'Is', 'All', 'You', 'Need', ':', 'A', 'Paradigm', 'Shift', 'in', 'Sequence', 'Modeling', 'The', 'year', '2017', 'marked', 'a', 'pivotal', 'moment', 'in', 'the', 'history', 'of', 'deep', 'learning', 'with', 'the', 'publication', 'of']


In [11]:
#Mycode
preprocessed = re.split(r'([,.:;?_!"()\']|\*|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['##', 'Attention', 'Is', 'All', 'You', 'Need', ':', 'A', 'Paradigm', 'Shift', 'in', 'Sequence', 'Modeling', 'The', 'year', '2017', 'marked', 'a', 'pivotal', 'moment', 'in', 'the', 'history', 'of', 'deep', 'learning', 'with', 'the', 'publication', 'of']


In [12]:
print(len(preprocessed))

1727


In [13]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

630


In [14]:
vocab = {token:integer for integer,token in enumerate(all_words)}

In [15]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('"', 0)
('##', 1)
('###', 2)
('####', 3)
("'", 4)
('(', 5)
(')', 6)
('*', 7)
(',', 8)
('-', 9)
('--', 10)
('.', 11)
('1', 12)
('2', 13)
('2017', 14)
('3', 15)
(':', 16)
(';', 17)
('A', 18)
('AI', 19)
('Addressing', 20)
('All', 21)
('Although', 22)
('Architecture', 23)
('Architectures', 24)
('Attention', 25)
('BERT', 26)
('Because', 27)
('Before', 28)
('Beyond', 29)
('Bias', 30)
('Brain', 31)
('By', 32)
('Challenges', 33)
('Computational', 34)
('Conclusion', 35)
('Consumption', 36)
('Context', 37)
('Cost', 38)
('Crucially', 39)
('Decoder', 40)
('Deconstructing', 41)
('Despite', 42)
('Directions', 43)
('Each', 44)
('Encoder', 45)
('Encoder-Decoder', 46)
('Encoding', 47)
('Energy', 48)
('Feed-Forward', 49)
('Fixed', 50)


In [16]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|\*|--|\s)', text)

        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations and asterisk
        text = re.sub(r'\s+([,.:;?_!"()\']|\*)', r'\1', text)
        # Replace spaces around the double hyphen
        text = re.sub(r'\s+(--)\s+', r'\1', text)
        return text

In [17]:
tokenizer = SimpleTokenizerV1(vocab)

text = """"Cost for Long Sequences:** The self-attention mechanism has a quadratic complexity with respect to the sequence length, meaning that as sequences get very long, the computational cost (both memory and processing time) becomes prohibitive. This is an active area of research, with efforts to develop more efficient attention mechanisms (e.g., sparse attention, linear attention, attention with local windows).
* **Interpretability:** While attention weights can."""
ids = tokenizer.encode(text)
print(ids)

[0, 38, 305, 82, 107, 16, 7, 7, 113, 525, 406, 326, 127, 480, 192, 622, 510, 581, 572, 529, 384, 8, 404, 571, 161, 530, 318, 605, 391, 8, 572, 195, 213, 5, 174, 408, 151, 470, 578, 6, 169, 473, 11, 115, 364, 149, 133, 158, 432, 505, 8, 622, 259, 581, 236, 414, 257, 165, 407, 5, 250, 11, 313, 11, 8, 545, 165, 8, 388, 165, 8, 165, 622, 390, 621, 6, 11, 7, 7, 7, 70, 16, 7, 7, 124, 165, 613, 181, 11]


In [18]:
tokenizer.decode(ids)

'" Cost for Long Sequences:** The self-attention mechanism has a quadratic complexity with respect to the sequence length, meaning that as sequences get very long, the computational cost( both memory and processing time) becomes prohibitive. This is an active area of research, with efforts to develop more efficient attention mechanisms( e. g., sparse attention, linear attention, attention with local windows).*** Interpretability:** While attention weights can.'

In [19]:
tokenizer.decode(tokenizer.encode(text))

'" Cost for Long Sequences:** The self-attention mechanism has a quadratic complexity with respect to the sequence length, meaning that as sequences get very long, the computational cost( both memory and processing time) becomes prohibitive. This is an active area of research, with efforts to develop more efficient attention mechanisms( e. g., sparse attention, linear attention, attention with local windows).*** Interpretability:** While attention weights can.'

In [20]:
try:
    tokenizer = SimpleTokenizerV1(vocab)
    text = "Hello, do you like tea. Is this-- a test?"
    tokenizer.encode(text)
except Exception as e:
    print("The error is shown intententionally")
    print(e)

The error is shown intententionally
'Hello'


In [21]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}

In [22]:
len(vocab.items())

632

In [23]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('work', 627)
('works', 628)
('year', 629)
('<|endoftext|>', 630)
('<|unk|>', 631)


In [24]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|\*|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations and asterisk
        text = re.sub(r'\s+([,.:;?_!"()\']|\*)', r'\1', text)
        # Replace spaces around the double hyphen
        text = re.sub(r'\s+(--)\s+', r'\1', text)
        return text

In [25]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [26]:
tokenizer.encode(text)

[631, 8, 631, 631, 385, 631, 631, 630, 67, 572, 631, 631, 432, 572, 631, 11]

In [27]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, <|unk|> <|unk|> like <|unk|> <|unk|> <|endoftext|> In the <|unk|> <|unk|> of the <|unk|>.'

In [28]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

tiktoken version: 0.9.0


In [29]:
tokenizer = tiktoken.get_encoding("gpt2")

In [30]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [31]:
strings = tokenizer.decode(integers)

print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


In [32]:
with open("sample_data.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

1979


In [33]:
enc_sample = enc_text[50:]

In [34]:
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")

x: [435, 13, 422, 3012]
y:      [13, 422, 3012, 14842]


In [35]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "---->", desired)

[435] ----> 13
[435, 13] ----> 422
[435, 13, 422] ----> 3012
[435, 13, 422, 3012] ----> 14842


In [36]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 al ----> .
 al. ---->  from
 al. from ---->  Google
 al. from Google ---->  Brain


In [37]:
import torch
print("PyTorch version:", torch.__version__)

PyTorch version: 2.6.0+cu124


In [38]:
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        assert len(token_ids) > max_length, "Number of tokenized inputs must at least be equal to max_length+1"

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [39]:
def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [40]:
with open("sample_data.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [41]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  198,  2235, 47406,  1148]]), tensor([[ 2235, 47406,  1148,  1439]])]


In [42]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 2235, 47406,  1148,  1439]]), tensor([[47406,  1148,  1439,   921]])]


In [43]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[  198,  2235, 47406,  1148],
        [ 1439,   921, 10664,    25],
        [  317, 14853, 17225, 15576],
        [  287, 45835,  9104,   278],
        [  198,   198,   464,   614],
        [ 2177,  7498,   257, 28992],
        [ 2589,   287,   262,  2106],
        [  286,  2769,  4673,   351]])

Targets:
 tensor([[ 2235, 47406,  1148,  1439],
        [  921, 10664,    25,   317],
        [14853, 17225, 15576,   287],
        [45835,  9104,   278,   198],
        [  198,   464,   614,  2177],
        [ 7498,   257, 28992,  2589],
        [  287,   262,  2106,   286],
        [ 2769,  4673,   351,   262]])


In [44]:
input_ids = torch.tensor([2, 3, 5, 1])

In [45]:
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [46]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [47]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


In [48]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


In [49]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [50]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [51]:
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[  198,  2235, 47406,  1148],
        [ 1439,   921, 10664,    25],
        [  317, 14853, 17225, 15576],
        [  287, 45835,  9104,   278],
        [  198,   198,   464,   614],
        [ 2177,  7498,   257, 28992],
        [ 2589,   287,   262,  2106],
        [  286,  2769,  4673,   351]])

Inputs shape:
 torch.Size([8, 4])


In [52]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

print(token_embeddings)

torch.Size([8, 4, 256])
tensor([[[-1.4371e-01, -9.7823e-01,  1.5918e+00,  ...,  3.0685e-01,
          -1.1135e+00, -7.2020e-01],
         [ 1.3515e+00,  8.6397e-01,  4.1770e-01,  ..., -3.9139e-01,
           9.5636e-02, -3.3542e-02],
         [-3.8940e-04,  8.2426e-01,  9.0248e-01,  ...,  4.5783e-01,
          -6.3047e-01, -2.1045e+00],
         [ 8.2668e-01,  4.9122e-01, -2.9964e-01,  ...,  9.0921e-01,
           8.7955e-01, -1.9174e-01]],

        [[ 1.6907e+00,  3.8105e-01,  1.9504e+00,  ..., -1.0960e+00,
           2.1464e-01,  1.5693e-01],
         [-3.6616e-02,  7.1712e-01,  4.4482e-02,  ..., -1.9617e+00,
          -1.0782e+00,  2.0892e+00],
         [ 1.1372e+00, -7.7334e-01,  1.5494e-01,  ..., -1.6836e-01,
           7.7261e-01, -4.6987e-01],
         [-8.4685e-02,  1.5248e+00,  1.6626e-01,  ...,  1.4983e-01,
          -7.4181e-01,  2.1374e+00]],

        [[ 1.9074e+00, -1.3086e-01,  1.5360e+00,  ...,  9.3841e-01,
           1.8633e+00,  5.6408e-01],
         [ 2.0162e-01, -2.5

In [53]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

print(pos_embedding_layer.weight)

Parameter containing:
tensor([[ 1.7375, -0.5620, -0.6303,  ..., -0.2277,  1.5748,  1.0345],
        [ 1.6423, -0.7201,  0.2062,  ...,  0.4118,  0.1498, -0.4628],
        [-0.4651, -0.7757,  0.5806,  ...,  1.4335, -0.4963,  0.8579],
        [-0.6754, -0.4628,  1.4323,  ...,  0.8139, -0.7088,  0.4827]],
       requires_grad=True)


In [54]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)
print(pos_embeddings)

torch.Size([4, 256])
tensor([[ 1.7375, -0.5620, -0.6303,  ..., -0.2277,  1.5748,  1.0345],
        [ 1.6423, -0.7201,  0.2062,  ...,  0.4118,  0.1498, -0.4628],
        [-0.4651, -0.7757,  0.5806,  ...,  1.4335, -0.4963,  0.8579],
        [-0.6754, -0.4628,  1.4323,  ...,  0.8139, -0.7088,  0.4827]],
       grad_fn=<EmbeddingBackward0>)


# Chapter 3: Implementing Self Attention